In [1]:

import random
import collections
import re
import requests
import random 
from torch.nn import functional as F
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
import sys
sys.path.append('../')
from utils.useful_func import *
import torch
from torch import nn
from torch.nn import functional as F

In [10]:
with open('../data/time_machine.txt','r') as f:
    content=f.readlines()
##读取文章
def read_time_machine():
    with open('../data/time_machine.txt','r') as f:
        content=f.readlines()
    return [ re.sub('[^A-Za-z]', ' ', i.replace('\n','')).strip().lower() for i in content ]

## 定义一个拆分词元的函数 结果是词元组成的list
def tokenize(content,token='word'):
    if token=='word':
        token_list=[token.lower() for i in content for token in i.split(' ')]
    else:
        token_list=[token.lower() for i in content for token in i]
    return token_list

##定义一个统计频率的函数 可以处理1d2d
def count_corpus(token_list):
    if isinstance(token_list[0], list):
      tokens=[token for i in token_list for token in i ]
    tokens_count=collections.Counter(token_list)
    return tokens_count

class Vocal():
    def __init__(self,token_list=None,min_feq=0,reserved_tokens=None):
        self.token_list=token_list
        if token_list is None:
            self.token_list=[]
        if reserved_tokens is None:
            reserved_tokens=[]
        counter_info=count_corpus(token_list)
        self._token_feq=[]
        ##只接受符合条件的词
        for items in sorted(counter_info.items(),key=lambda x:x[1],reverse=True):
            if items[1]<=min_feq:
                break
            else:
                self._token_feq.append(items)

        ##
        if '<unk>' in reserved_tokens:
            self.idx_to_token=reserved_tokens
        else:
            self.idx_to_token=['unk']+reserved_tokens

        self.token_to_idx={token:i for i,token in enumerate(self.idx_to_token)}
        for token,_ in self._token_feq:
            self.idx_to_token.append(token)
            self.token_to_idx[token]=len(self.token_to_idx)
    def __len__(self):
        return len(self.token_to_idx)
    ##实现一个索引方法 但是传入的索引是token 返回是idx,保证未知token显示0值
    def __getitem__(self,tokens):
        if not isinstance(tokens,(tuple,list)):
            return self.token_to_idx.get(tokens,0)
        return [self.__getitem__(i) for i in tokens]

    @property
    def unk(self):
        return 0

    @property
    def tokens_freq(self):
        return self._token_feq

def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    tokens=tokenize(read_time_machine(),'char')
    vocal=Vocal(tokens)

    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocal[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocal

##产生随机序列
def seq_data_iter_random(corpus,batch_size,num_steps):
    """使用随机抽样生成一个小批量子序列"""
    ##randint是左闭右闭 所以必须得减一不然0和numsteps实际上是重复的
    corpus=corpus[random.randint(0,num_steps-1):]
    ##因为y要比x多一位 一共可以有这么多子序列
    num_subseqs=(len(corpus)-1)//num_steps

    ## 然后找出每一个num seq的起始索引拿出来放到列表里 这里不能用corpus的长度 而是num_steps*num_subseqs
    seq_start_index=[i for i in range(0,num_steps*num_subseqs,num_steps)]
    ##打乱索引
    random.shuffle(seq_start_index)

    ## 再除以batchsize 有多少个batch
    num_batch=num_subseqs//batch_size
    ### 然后给每个batch找x和y 索引起始位置在seq_start_index里
    for i in range(num_batch):
        index_list=seq_start_index[i*batch_size:(i+1)*batch_size]
        x=[]
        y=[]
        for _index in index_list:
           x.append(corpus[_index:_index+num_steps])
           y.append(corpus[_index+1:_index+1+num_steps])
        yield torch.tensor(x),torch.tensor(y)
        ##结果是2维的，dim=0 是batchsize dim=1是时间步

# def seq_data_iter_sequential(corpus,batch_size,num_steps):
#     random.seed(42)
#     corpus=corpus[random.randint(0,num_steps):]
#     batch_num=(len(corpus)-1)//(batch_size*num_steps)
#     Xs=torch.tensor(corpus[:batch_num*batch_size*num_steps]).reshape(-1,batch_size,num_steps)
#     Ys=torch.tensor(corpus[1:batch_num*batch_size*num_steps+1]).reshape(-1,batch_size,num_steps)
#     for i in range(batch_num):
#         yield Xs[i],Ys[i]
# s1=seq_data_iter_sequential(corpus,batch_size,num_steps)
# 这个方案是错的 连续要求在不同的batch上保持连续 而不是在一个batch的多个样本上保持连续 并且 batch_num=(len(corpus)-1)//(batch_size*num_steps) 这样也不太好
#因为batch_num 计算方式耦合了 batch_size 和 num_steps，


def seq_data_iter_sequential(corpus,batch_size,num_steps):
    random.seed(0)
    corpus=corpus[random.randint(0,num_steps):]
    num_tokens=(len(corpus)-1)//batch_size*batch_size ## 保证是batch_size的倍数
    ###batch_size需要放在最外维度：常规样本维度安排 外层是样本个数 也就是batch_size
    Xs=torch.tensor(corpus[:num_tokens]).reshape(batch_size,-1)
    Ys=torch.tensor(corpus[1:num_tokens+1]).reshape(batch_size,-1)
    ##维度1是每个batchsize有多少token
    batch_num=Xs.shape[1]//num_steps
    for i in range(batch_num):
        yield Xs[:,i*num_steps:(i+1)*num_steps],Ys[:,i*num_steps:(i+1)*num_steps]
class SeqDataLoader:  #@save
    def __init__(self,batch_size,num_steps,use_random_iter,max_token):
        if use_random_iter:
            self.data_iter_fn=seq_data_iter_random
        else:
            self.data_iter_fn=seq_data_iter_sequential
        self.corpus,self.vocal=load_corpus_time_machine(max_token)
        self.batch_size,self.num_steps = batch_size,num_steps

    ##__iter__方法使得整个类变成可迭代的
    def __iter__(self):
        return self.data_iter_fn(self.corpus,self.batch_size,self.num_steps)

def load_data_time_machine(batch_size,num_steps,use_random_iter=False,max_token=10000):
    data_iter=SeqDataLoader(batch_size,num_steps,use_random_iter,max_token)
    return data_iter,data_iter.vocal


In [11]:
batch_size, num_steps = 32, 35
num_hidden=512
train_iter, vocab = load_data_time_machine(batch_size, num_steps)

In [12]:

rnn_layer=nn.RNN(len(vocab),num_hidden)
##定义循环神经网络
class RNN(nn.Module):
    def __init__(self,rnn_layer,vocab_size, **kwargs):
        super(RNN,self).__init__(**kwargs)
        self.rnn=rnn_layer
        self.vocab_size=vocab_size
        self.hidden_size=self.rnn.hidden_size
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear=nn.Linear(self.hidden_size,self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)
    def forward(self,input,state):
        X = F.one_hot(input.T.long(),self.vocab_size)
        ##nn.RNN会自动预热
        Y,state=self.rnn(X.to(dtype=torch.float32),state) ##输出维度是（x,num_steps,hiddens）
        output=self.linear(Y.reshape(-1,Y.shape[-1]))
        return output,state
    ##初始化隐状态
    def begin_state(self,device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            state=torch.zeros((self.num_directions * self.rnn.num_layers,batch_size,self.hidden_size),device=device)
            return  state
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((
                        self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens), device=device))

def predict_ch8(prefix, num_preds, net, vocab, device):  #@save
    """在prefix后面生成新字符"""
    state = net.begin_state(batch_size=1, device=device)
    ##outputs只有第一个字母
    outputs = [vocab[prefix[0]]]
    ## 定义一个获取最新output的函数
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期
        ##get_input()每次只有一个值 作为输入 state为隐状态
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

def grad_clipping(net,theta):
    if isinstance(net,nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    # norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    norm=torch.sqrt(torch.sum(torch.tensor([torch.sum(p.grad**2) for p in params])))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm


def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    state, timer = None, Timer()
    metric = Accumulator(2)
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 初始化状态
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            # 非原地分离状态
            if isinstance(net, nn.Module):
                if isinstance(state, tuple):
                    # LSTM状态：分离每个张量
                    state = tuple(s.detach() for s in state)
                else:
                    # GRU状态：直接分离
                    state = state.detach()
            else:
                # 自定义模型状态处理
                state = (s.detach() for s in state)
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

#@save
def train_ch8(net, train_iter, vocab, lr, num_epochs, device,
              use_random_iter=False):
    """训练模型（定义见第8章）"""
    loss = nn.CrossEntropyLoss()

    # 初始化
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
    # 训练和预测
    for epoch in range(num_epochs):
        ppl, speed = train_epoch_ch8(
            net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict('time traveller'))
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    print(predict('time traveller'))
    print(predict('traveller'))


In [14]:
net=RNN(rnn_layer,len(vocab))
net=net.to(torch.device('cuda:0'))

In [15]:
predict_ch8('time traveller', 10, net, vocab, 'cuda:0')

'time travellerzgzgzgzgzg'

In [16]:
num_epochs, lr = 500, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, 'cuda:0')

time traveller  an  an  an  an  an  an  an  an  an  an  an  an  
time traveller  and the  and the  and the  and the  and the  and
time traveller  four  in the time traveller  four   and the  the
time traveller  and the tre  the the the the the the the the the
time traveller  mat  ally the time t an t an t an   ar and the t
time traveller  wall  spanced me time tre  lan   and and and and
time traveller ally ally there andion ancagoully ascentareedithe
time traveller  and ther the timent on tor s an sour than tory t
time traveller  you gand the paydon t fall the sime sime sacint 
time traveller  matind ve y on this tare  beens and the pacent s
time traveller said the psoghthe pseif chith and sore that usera
time traveller  mathe  berineto the s mans move about   soid the
time traveller  tha   are ald he right   avely owey lople   ve i
time travellery mone dime sion d mint an thertimint thertert med
time traveller   for an wio  ouk and son t widthe pucknt t me in
time travellere mitthree 

In [65]:
net=nn.Linear(1,1)

In [66]:
x1=torch.tensor(2)

In [67]:
x=torch.ones(1,1)
x.requires_grad=True
y=x*x1
y.backward()
x.grad

tensor([[2.]])

In [68]:
x.detach_()
x.requires_grad

False

In [72]:
x=net(x)
x.requires_grad

TypeError: Linear.forward() takes 2 positional arguments but 3 were given

In [60]:
x=x*x1
y_2=x*x1
y_2.backward()
x1.grad

False

tensor([[5.]])